This is an example of using RNN and LSTM to predict the next value in a time series.

In [ ]:
# Import pandas and give it an alias pd
import pandas as pd

In [ ]:
# We read in the CSV file, selecting only the second column (passenger numbers), using the Python engine to 
# traverse the data
dataframe = pd.read_csv('airline-passengers.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

In [ ]:
# import matplotlib for plotting and visualisation
import matplotlib.pyplot as plt

# plot and display original dataset (output below)
plt.plot(dataset)
plt.show()

In [ ]:
# Here, we normalize the dataset to a range of (0,1) for better performance during the model's training. 
# This ensures that all values are scaled between 0 and 1, which is helpful for the neural network
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
# Split the dataset into training and test sets.
# train_ratio defines the proportion of data to be used for training (67% in this case).
train_ratio = 0.67
train_ratio = 0.67
train_size = int(len(dataset) * train_ratio)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
'''
Task: After you run this notebook successfuly, change train_ratio to be 0.80, and run the notebook again, record the evaluation metrics and then compare the results
'''

In [ ]:
# Print the sizes of training and testing sets to verify the split.
print(len(train), len(test))

In [ ]:
# The create_dataset function is used to transform a given dataset into a format that's suitable for time series forecasting.
# The look_back parameter specifies how many previous steps to use as input features for the prediction at the next time step.
# For example, when we have a dataset = [[1], [2], [3], [4], [5], [6]] and look_back is 1,
# dataX will be [[1], [2], [3], [4], [5]]
# dataY will be [2, 3, 4, 5, 6]
# So the model will be trained to predict 2 given [1], 3 given [2], 4 given [3], 5 given [4], and 6 given [5].

# This function transforms the dataset into a format suitable for time series prediction.
# 'look_back' specifies how many previous time steps are used to predict the next value.
# The function returns dataX (input features) and dataY (labels or target values).
import numpy as np

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a) # store the input sequence
        b = dataset[i + look_back, 0] # output data (next value)
        dataY.append(b) # store the output (label) sequence
    return np.array(dataX), np.array(dataY)

In [ ]:
# reshape into X=t and Y=t+1
look_back = 1 # Set look_back to 1, which means that we'll use one previous time step to predict the next one.
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
# This prints the training input and corresponding labels verifying the format
for i in range (len(trainX)):
    print(trainX[i], trainY[i])

In [ ]:
# We reshape the input data to be in the format [samples, time steps, features], a required stepf for RNN models.
# Here, '1' represents the time steps (since we're using a single time step for each sample).
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

Input a model code, `model_code = 1` for RNN and `model_code = 2` for LSTM.

In [ ]:
model_code = int(input("Input a model code, model_code = 1 for RNN and model_code = 2 for LSTM. Your choice: "))

In [ ]:
# Set up parameters for running the model
epochNo = 100
batchSize = 1

In [ ]:
# create and fit the RNN or LSTM network
# import required packages
import tensorflow as tf
from tensorflow.keras.layers import Dense

if (model_code == 1):
    '''
    Task: import SimpleRNN from tensorflow.keras.layers for using RNN model
    # '''
    # We're building a simple RNN model using the SimpleRNN layer from TensorFlow/Keras.
    # RNNs are designed to process sequential data where previous information influences the current output.
    # In each time step, the RNN processes one input (trainX), updates its hidden state, and generates an output.
    # The RNN "remembers" information from previous steps using its hidden state, which is carried from one step to the next.
    # The hidden state allows the model to consider prior time steps when making predictions about the next time step.
    # For example, if the input data represents passenger numbers in the past, the RNN uses the hidden state to store information
    # about previous passenger trends to predict the number of passengers in the next step.
    # In this case, we add one SimpleRNN layer with 4 neurons/units, which means the RNN will have 4 different memory states

    from tensorflow.keras.layers import SimpleRNN


    model = tf.keras.Sequential()
    '''
    Task: Add a SimpleRNN layer using method add() with parameters: (4,input_shape=(1, look_back)) for the SimpleRNN layer
    '''
    
    model.add(SimpleRNN(4,input_shape=(1, look_back)))
    model.add(Dense(1))


    '''
    Task: Compile the model using method compile() from the model with parameters: loss='mean_squared_error', optimizer='adam'
    '''
    model.compile(loss='mean_squared_error', optimizer='adam')


    '''
    Task: Train the model using method fit() from the model with the tran dataset, epochs=epochNo, batch_size=batchSize, verbose=2
    '''
    "Add your code here"
    model.fit(trainX, trainY, epochs=epochNo, batch_size=batchSize, verbose=2)


elif (model_code == 2):
    '''
    Task: import LSTM from tensorflow.keras.layers for using LSTM model
    '''
    "Add your code here"
    
    from tensorflow.keras.layers import LSTM


    model =tf.keras.Sequential()
    '''
    Task: Add a LSTM layer using method add() with parameters: (4, input_shape=(1, look_back)) for the LSTM layer
    '''
    "Add your code here"
    
    model.add(LSTM(4, input_shape=(1, look_back)))


    model.add(Dense(1))
    
    '''
    Task: Compile the model using method compile() from the model with parameters: loss='mean_squared_error', optimizer='adam'
    '''
    "Add your code here"
    
    model.compile(loss='mean_squared_error', optimizer='adam')
  
    '''
    Task: Train the model using method fit() from the model with the tran dataset, epochs=epochNo, batch_size=batchSize, verbose=2
    '''
    "Add your code here"
    
    model.fit(trainX, trainY, epochs=epochNo, batch_size=batchSize, verbose=2)

else:
    print("invalid model code, try a correct code, 1 or 2")

In [ ]:
# Predict the train set using the trained model and invert the scaling to its original values.
trainPredict = model.predict(trainX)

In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])

In [ ]:
# make predictions of test set
testPredict = model.predict(testX)

In [ ]:
# invert predictions
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

### Evalutate the model's prediction
We calculate and print the RMSE for the test set to evaluate how well the model generalised to its unseen data.

In [ ]:
# calculate root mean squared error and import required package
from sklearn.metrics import mean_squared_error

# The RMSE (Root Mean Squared Error) is a widely used metric to evaluate the performance of regression models.
# It provides a measure of the average error (or deviation) between the predicted values and the actual values.
# In this case, the model's predictions (trainPredict and testPredict) are compared to the actual passenger numbers (trainY and testY).

# NOTE THAT: a lower RMSE indicates that the model's predictions are closer to the actual values, meaning the model performs well.
# In contrast, a higher RMSE suggests that the model's predictions are less accurate. 
# By calculating RMSE for both the training and test sets, we can evaluate how well the model has learned 
# and whether it generalizes well to unseen data (the test set).
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#### Plot the original dataset, the train predictions, and the test predictions for visualization.

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot) # train set predictions
plt.plot(testPredictPlot) # test set predictions
plt.show() # display them altogether!